In [112]:
import pandas as pd
import numpy as np

In [113]:
df = pd.read_csv('nba odds 2020-21.csv').dropna(axis=1)

In [114]:
df

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,1222,501,V,GoldenState,25,20,26,28,99,228,234.5,265,1
1,1222,502,H,Brooklyn,40,23,36,26,125,5.5,7.5,-320,115
2,1222,503,V,LAClippers,39,17,33,27,116,223,219,115,108.5
3,1222,504,H,LALakers,19,35,24,31,109,2.5,2,-135,3.5
4,1223,551,V,Charlotte,23,21,36,34,114,215,3,-140,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,714,508,H,Milwaukee,20,32,24,33,109,3.5,4.5,-190,3
2338,717,509,V,Milwaukee,21,43,36,23,123,218.5,220,150,112.5
2339,717,510,H,Phoenix,37,24,29,29,119,4,4,-170,4
2340,720,511,V,Phoenix,16,31,30,21,98,223,221,170,112.5


In [115]:
drop_cols = ['index', '2H', 'VH', 'Rot', '1st', '2nd', '3rd', '4th']
df_v = df[df['VH'] == 'V'].reset_index().drop(drop_cols, axis=1)
df_h = df[df['VH'] == 'H'].reset_index().drop(drop_cols + ['Date'], axis=1)
df_h

,Team,Final,Open,Close,ML
0,Brooklyn,125,5.5,7.5,-320
1,LALakers,109,2.5,2,-135
2,Cleveland,121,1,217,120
3,Indiana,121,8.5,7.5,-330
4,Orlando,113,220.5,219,170
...,...,...,...,...,...
1166,Phoenix,118,5.5,4.5,-180
1167,Milwaukee,120,3.5,4.5,-210
1168,Milwaukee,109,3.5,4.5,-190
1169,Phoenix,119,4,4,-170


In [116]:
df1 = pd.concat([df_v, df_h], axis=1)
df1.columns = ['Date', 'v', 'final_v', 'open1', 'close1', 'ml_v', 'h', 'final_h', 'open2', 'close2', 'ml_h']

df1[['open1', 'open2']] = df1[['open1', 'open2']].replace({'pk': 0})
df1[['close1', 'close2']] = df1[['close1', 'close2']].replace({'pk': 0})

df1[['open1', 'open2', 'close1', 'close2']] = df1[['open1', 'open2', 'close1', 'close2']].astype(float)

df1['total_open'] = df1[['open1', 'open2']].max(axis=1)
df1['total_close'] = df1[['close1', 'close2']].max(axis=1)
df1['spread_open'] = df1[['open1', 'open2']].min(axis=1)
df1['spread_close'] = df1[['close1', 'close2']].min(axis=1)

df1['spread_open'] = np.where(df1['open1'] > df1['open2'], 
                              df1['spread_open']*-1,
                              df1['spread_open'])

df1['spread_close'] = np.where(df1['close1'] > df1['close2'], 
                              df1['spread_close']*-1,
                              df1['spread_close'])

df1 = df1[(df1['total_open'] > 100) & (df1['total_close'] > 100)]
display(df1)
df1 = df1.dropna()
df1 = df1.drop(['open1', 'close1', 'open2', 'close2'], axis=1)
df1

,Date,v,final_v,open1,close1,ml_v,h,final_h,open2,close2,ml_h,total_open,total_close,spread_open,spread_close
0,1222,GoldenState,99,228.0,234.5,265,Brooklyn,125,5.5,7.5,-320,228.0,234.5,-5.5,-7.5
1,1222,LAClippers,116,223.0,219.0,115,LALakers,109,2.5,2.0,-135,223.0,219.0,-2.5,-2.0
2,1223,Charlotte,114,215.0,3.0,-140,Cleveland,121,1.0,217.0,120,215.0,217.0,-1.0,3.0
3,1223,NewYork,107,214.5,215.5,270,Indiana,121,8.5,7.5,-330,214.5,215.5,-8.5,-7.5
4,1223,Miami,107,3.5,4.5,-190,Orlando,113,220.5,219.0,170,220.5,219.0,3.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,708,Milwaukee,108,221.0,221.0,160,Phoenix,118,5.5,4.5,-180,221.0,221.0,-5.5,-4.5
1167,711,Phoenix,100,222.0,220.0,180,Milwaukee,120,3.5,4.5,-210,222.0,220.0,-3.5,-4.5
1168,714,Phoenix,103,220.5,221.0,170,Milwaukee,109,3.5,4.5,-190,220.5,221.0,-3.5,-4.5
1169,717,Milwaukee,123,218.5,220.0,150,Phoenix,119,4.0,4.0,-170,218.5,220.0,-4.0,-4.0


,Date,v,final_v,ml_v,h,final_h,ml_h,total_open,total_close,spread_open,spread_close
0,1222,GoldenState,99,265,Brooklyn,125,-320,228.0,234.5,-5.5,-7.5
1,1222,LAClippers,116,115,LALakers,109,-135,223.0,219.0,-2.5,-2.0
2,1223,Charlotte,114,-140,Cleveland,121,120,215.0,217.0,-1.0,3.0
3,1223,NewYork,107,270,Indiana,121,-330,214.5,215.5,-8.5,-7.5
4,1223,Miami,107,-190,Orlando,113,170,220.5,219.0,3.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...
1166,708,Milwaukee,108,160,Phoenix,118,-180,221.0,221.0,-5.5,-4.5
1167,711,Phoenix,100,180,Milwaukee,120,-210,222.0,220.0,-3.5,-4.5
1168,714,Phoenix,103,170,Milwaukee,109,-190,220.5,221.0,-3.5,-4.5
1169,717,Milwaukee,123,150,Phoenix,119,-170,218.5,220.0,-4.0,-4.0


In [117]:
df1.to_csv('2020-21_odds_parsed.csv')